In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def weight_var(shape, dropout=True, stddev=0.05):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    
    return var
def bias(shape):
    return tf.Variable(tf.constant(0.01, shape=shape))

In [4]:
sess = tf.InteractiveSession()

In [5]:
X = tf.placeholder(tf.float32, [None, 784])

D_w1 = weight_var([784, 128])
D_b1 = bias([128])

D_w2 = weight_var([128, 1])
D_b2 = bias([1])

def discrim(x):
    D_out1 = tf.nn.relu(tf.matmul(x, D_w1) + D_b1)
    D_out2 = tf.matmul(D_out1, D_w2) + D_b2
    D_probs = tf.nn.sigmoid(D_out2)
    
    return D_out2, D_probs

D_vars = [D_w1, D_b1, D_w2, D_b2]

In [6]:
Z = tf.placeholder(tf.float32, [None, 100])

G_w1 = weight_var([100, 128])
G_b1 = bias([128])

G_out1 = tf.nn.relu(tf.matmul(Z, G_w1) + G_b1)

G_w2 = weight_var([128, 784])
G_b2 = bias([784])

G_out2 = tf.matmul(G_out1, G_w2) + G_b2

G_sample = tf.nn.sigmoid(G_out2)
G_sample_image = tf.summary.image("generator", tf.reshape(G_sample, [-1, 28, 28, 1]), 3)


G_vars = [G_w1, G_b1, G_w2, G_b2]

In [7]:
D_real_logits, D_real = discrim(X)
D_fake_logits, D_fake = discrim(G_sample)

In [8]:
D_real_loss = tf.log(tf.clip_by_value(D_real, 1e-10, 1.0))
D_fake_loss = tf.log(tf.clip_by_value(1. - D_fake, 1e-10, 1.0))



D_loss = -tf.reduce_mean(D_real_loss + D_fake_loss)
G_loss = -tf.reduce_mean(tf.log(tf.clip_by_value(D_fake, 1e-10, 1.0)))

In [9]:
D_trainer = tf.train.AdamOptimizer(1e-4).minimize(D_loss, var_list=D_vars)
G_trainer = tf.train.AdamOptimizer(1e-4).minimize(G_loss, var_list=G_vars)

In [10]:
tf.global_variables_initializer().run()

summary_writer = tf.summary.FileWriter("tmp/logs", sess.graph)

In [11]:
def sample_Z(batch_size):
    return np.random.uniform(-1, 1, [batch_size, 100])

np.random.seed(0xDEADBEEF)

In [14]:
BATCH_SIZE = 1000

for ITER in range(1000000):
    batch, _ = mnist.train.next_batch(BATCH_SIZE)
    fake_batch = sample_Z(BATCH_SIZE)
    
    D_trainer.run(feed_dict={Z: fake_batch, X: batch})
    G_trainer.run(feed_dict={Z: fake_batch, X: batch})
    
    if ITER % 5000 == 0:
        D_train_loss = D_loss.eval(feed_dict={Z: fake_batch, X: batch})
        G_train_loss = G_loss.eval(feed_dict={Z: fake_batch, X: batch})
        
        print("Iter %d: Losses: %.2f and %.2f"%(ITER, D_train_loss, G_train_loss))
        
        summary_writer.add_summary(G_sample_image.eval(feed_dict={Z: fake_batch, X: batch}), ITER)
        summary_writer.flush()
    

Iter 0: Losses: 0.61 and 1.99
Iter 5000: Losses: 0.66 and 1.85
Iter 10000: Losses: 0.57 and 2.20
Iter 15000: Losses: 0.57 and 2.11
Iter 20000: Losses: 0.52 and 2.52
Iter 25000: Losses: 0.73 and 1.83
Iter 30000: Losses: 0.56 and 2.16
Iter 35000: Losses: 0.48 and 2.61
Iter 40000: Losses: 0.48 and 2.71
Iter 45000: Losses: 0.55 and 2.31
Iter 50000: Losses: 0.71 and 2.12
Iter 55000: Losses: 0.59 and 2.18
Iter 60000: Losses: 0.51 and 2.53
Iter 65000: Losses: 0.65 and 2.04
Iter 70000: Losses: 1.14 and 1.23
Iter 75000: Losses: 0.60 and 2.06
Iter 80000: Losses: 0.54 and 2.46
Iter 85000: Losses: 1.40 and 1.37
Iter 90000: Losses: 0.50 and 2.41
Iter 95000: Losses: 0.76 and 1.87
Iter 100000: Losses: 0.70 and 1.81
Iter 105000: Losses: 0.80 and 1.86
Iter 110000: Losses: 0.63 and 2.12
Iter 115000: Losses: 0.52 and 2.34
Iter 120000: Losses: 0.62 and 2.17
Iter 125000: Losses: 0.55 and 2.41
Iter 130000: Losses: 0.65 and 2.38
Iter 135000: Losses: 1.79 and 1.28
Iter 140000: Losses: 0.58 and 2.28
Iter 14500